In [1]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdf2casadi.urdfparser
import urdf2casadi.urdf2casadi.geometry.plucker as plucker
import numpy as np
import PyKDL as kdl
import kdl_parser.kdl_parser_py.kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl

ImportError: No module named urdf2casadi.urdfparser

## Load models:

In [20]:
ok, ur_tree = kdlurdf.treeFromFile('./urdf2casadi/examples/urdf/test.urdf')
#ok, ur_tree = kdlurdf.treeFromFile('pantilt.urdf')
asd = urdf2casadi.urdf2casadi.urdfparser.URDFparser()
robot_desc = asd.from_file("./urdf2casadi/examples/urdf/test.urdf")
#robot_desc = asd.from_file("pantilt.urdf")
root = 'base_link'
tip = 'wrist_3_link'
ur_chain = ur_tree.getChain(root,tip)
rbdlmodel = rbdl.loadModel("ur5_rbdl.urdf")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
./urdf2casadi/examples/urdf/test.urdf


## Declare variables to use in timing tests:

In [22]:
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
print njoints
print len(qmax)

H_ur = kdl.JntSpaceInertiaMatrix(6)

grav = kdl.Vector()
grav[2] = -9.81

q = kdl.JntArray(njoints)
qdot = kdl.JntArray(njoints)

q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)

q_sx = cs.SX.zeros(njoints)
print q_sx
qdot_sx = cs.SX.zeros(njoints)

q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints

tau_kdl = kdl.JntArray(njoints)
grav_kdl = kdl.JntArray(njoints)

nitr = 50000
nrepeat = 3

M_rbdl = (rbdlmodel.q_size, rbdlmodel.q_size)
M_rbdl = np.zeros(M_rbdl)


6
6
@1=0, [@1, @1, @1, @1, @1, @1]


# Inertia Matrix Timing 
## KDL vs u2c:
With timeit module:

In [10]:
def M_kdl():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(ur_chain, grav).JntToMass(q, H_ur)

timeit_M_kdl = timeit("M_kdl()", setup = "from __main__ import M_kdl", number = nitr)
print timeit_M_kdl
print timeit_M_kdl/nitr
timeit_M_kdl_repeat = repeat("M_kdl()", setup = "from __main__ import M_kdl", repeat = 10, number = nitr)
print timeit_M_kdl_repeat

0.994987010956
1.98997402191e-05
[0.8941152095794678, 0.8484561443328857, 0.8353838920593262, 0.8279831409454346, 0.827606201171875, 0.8432409763336182, 0.8361051082611084, 0.8390989303588867, 0.8260359764099121, 0.8260140419006348]


In [8]:
M = asd.get_jointspace_inertia_matrix(root, tip)

In [9]:
def M_u2c():   
    for j in range(njoints):
        q_none = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    M(q_none)

timeit_M_u2c = timeit("M_u2c()", setup = "from __main__ import M_u2c", number = 50000)
print timeit_M_u2c
print timeit_M_u2c/50000
timeit_M_u2c_repeat = repeat("M_u2c()", setup = "from __main__ import M_u2c", repeat = 10, number = 50000)
print timeit_M_u2c_repeat

0.697988986969
1.39597797394e-05
[1.100003957748413, 0.8199138641357422, 0.6107070446014404, 0.6007380485534668, 0.5877580642700195, 0.5841989517211914, 1.0059311389923096, 0.9312691688537598, 1.1127619743347168, 0.5959780216217041]


With timeit magic:

In [25]:
def M_rbdl_func():
    for j in range(njoints):
        q_np[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    rbdl.CompositeRigidBodyAlgorithm(rbdlmodel, q_np, M_rbdl)
timeit_M_rbdl_repeat = repeat("M_rbdl_func()", setup = "from __main__ import M_rbdl_func", repeat = 10, number = 50000)
print timeit_M_rbdl_repeat

[2.5185980796813965, 2.3144681453704834, 2.453153133392334, 2.3757059574127197, 2.4539759159088135, 2.341848134994507, 2.374173879623413, 2.3893380165100098, 2.3690850734710693, 2.3069770336151123]
